In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.distributions as D
from diffusers.models.autoencoders.autoencoder_kl import AutoencoderKL
import torchvision

import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

sns.set_theme(style="dark")

In [2]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device("cuda:1")
    # if torch.backends.mps.is_available():
        # return torch.device("mps")
    return torch.device("cpu")

DEVICE = get_default_device()

In [3]:
from typing import cast

vae = AutoencoderKL.from_pretrained("stabilityai/sdxl-vae").to(DEVICE)
# vae = cast(AutoencoderKL, torch.compile(vae, mode="max-autotune"))
vae.eval()
pass

In [4]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomRotation(10),
])

In [5]:
class CifarDataset(torch.utils.data.Dataset):
    def __init__(self, train: bool = True):
        self.cifar = torchvision.datasets.CIFAR10(root="/tmp/cifar", download=True, train=train, transform=transform)

    def __len__(self):
        return len(self.cifar)

    def __getitem__(self, idx: int):
        x, y = self.cifar[idx]
        t = torch.rand(1)
        return x, y, t

In [6]:
cifar_train = CifarDataset(train=True)
cifar_test = CifarDataset(train=False)

train_dataloader = torch.utils.data.DataLoader(cifar_train, batch_size=256, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(cifar_test, batch_size=256, shuffle=True)

latent_shape = vae.encode(next(iter(train_dataloader))[0].to(DEVICE), return_dict=False)[0].mean[0].shape

next(iter(train_dataloader))[0].shape, next(iter(train_dataloader))[1].shape, next(iter(train_dataloader))[2].shape, latent_shape

(torch.Size([256, 3, 32, 32]),
 torch.Size([256]),
 torch.Size([256, 1]),
 torch.Size([4, 4, 4]))

In [7]:
class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
    def forward(self, x):
        half_dim = self.dim // 2
        emb = torch.exp(-torch.arange(half_dim, device=x.device) * torch.log(torch.tensor(10000.0)) / (half_dim - 1))
        emb = x.unsqueeze(1) * emb.unsqueeze(0)
        return torch.cat([emb.sin(), emb.cos()], dim=-1)

class ResidualBlock(nn.Module):
    def __init__(self, in_ch, out_ch, time_emb_dim, groups=8):
        super().__init__()
        self.time_mlp = nn.Sequential(
            nn.SiLU(),
            nn.Linear(time_emb_dim, out_ch)
        )
        self.conv_block = nn.Sequential(
            nn.GroupNorm(groups, in_ch),
            nn.SiLU(),
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.GroupNorm(groups, out_ch),
            nn.SiLU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
        )
        self.res_conv = nn.Conv2d(in_ch, out_ch, 1) if in_ch != out_ch else nn.Identity()

    def forward(self, x, t_emb):
        h = self.conv_block(x)
        time_proj = self.time_mlp(t_emb).unsqueeze(-1).unsqueeze(-1)
        h = h + time_proj
        return h + self.res_conv(x)

class UNet(nn.Module):
    def __init__(self, in_ch=4, base_ch=64, channel_mults=(1, 2), time_emb_dim=128):
        super().__init__()
        self.time_mlp = nn.Sequential(
            SinusoidalPosEmb(time_emb_dim),
            nn.Linear(time_emb_dim, time_emb_dim * 4),
            nn.SiLU(),
            nn.Linear(time_emb_dim * 4, time_emb_dim),
        )
        self.init_conv = nn.Conv2d(in_ch, base_ch, 3, padding=1)
        down_chs = [base_ch * m for m in channel_mults]
        self.downs = nn.ModuleList()
        prev_ch = base_ch
        for ch in down_chs:
            self.downs.append(ResidualBlock(prev_ch, ch, time_emb_dim))
            prev_ch = ch
        self.bottleneck = ResidualBlock(prev_ch, prev_ch, time_emb_dim)
        self.ups = nn.ModuleList()
        for idx, ch in enumerate(reversed(down_chs)):
            # ch is skip channel
            out_ch = down_chs[-2 - idx] if idx < len(down_chs) - 1 else base_ch
            self.ups.append(nn.ModuleDict({
                'upsample': nn.ConvTranspose2d(prev_ch, out_ch, 4, 2, 1),
                'res': ResidualBlock(out_ch + ch, out_ch, time_emb_dim)
            }))
            prev_ch = out_ch
        self.final_conv = nn.Sequential(
            nn.GroupNorm(8, base_ch),
            nn.SiLU(),
            nn.Conv2d(base_ch, in_ch, 1),
        )

    def forward(self, x, t):
        # x: (B, C, H, W), t: (B,)
        t_emb = self.time_mlp(t)
        h = self.init_conv(x)
        skips = []
        for block in self.downs:
            h = block(h, t_emb)
            skips.append(h)
            h = F.avg_pool2d(h, 2)
        h = self.bottleneck(h, t_emb)
        for module in self.ups:
            module = cast(nn.ModuleDict, module)
            h = module['upsample'](h)
            skip = skips.pop()
            h = torch.cat([h, skip], dim=1)
            h = module['res'](h, t_emb)
        # final
        return self.final_conv(h)

    def ode_forward(self, t: torch.Tensor, x: torch.Tensor):
        t_tensor = t.repeat(x.shape[0])
        return self.forward(x, t_tensor)

In [8]:
N_EPOCHS = 200

ckpt_path = "../ckpt/UNet"

model = UNet()
model = model.to(DEVICE)
optimizer = AdamW(model.parameters(), lr=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=N_EPOCHS)

if ckpt_path and False:
    model.load_state_dict(torch.load(f"{ckpt_path}/model.pth", map_location=DEVICE))
    optimizer.load_state_dict(torch.load(f"{ckpt_path}/optimizer.pth", map_location=DEVICE))
    scheduler.load_state_dict(torch.load(f"{ckpt_path}/scheduler.pth", map_location=DEVICE))

In [9]:
from torchdiffeq import odeint


@torch.no_grad()
def sample_with_ode(
    model: nn.Module,
    n_samples: int = 500,
    n_steps: int = 25,
):
    model.eval()
    model_device = next(model.parameters()).device
    initial_samples = torch.randn((n_samples, *latent_shape), device=model_device)
    t_span = torch.linspace(0.0, 1.0, n_steps).to(model_device)
    trajectory = odeint(
        model.ode_forward,
        initial_samples,
        t_span,
        method="euler",
        atol=1e-5,
        rtol=1e-5,
    )
    trajectory = cast(torch.Tensor, trajectory)

    return trajectory[-1]

In [10]:
from torchmetrics.image.fid import FrechetInceptionDistance
import os

def to_latent(vae: AutoencoderKL, x_raw: torch.Tensor) -> torch.Tensor:
    return vae.encode(x_raw, return_dict=False)[0].mean


def from_latent(vae: AutoencoderKL, z: torch.Tensor) -> torch.Tensor:
    return vae.decode(cast(torch.FloatTensor, z), return_dict=False)[0]

In [11]:
def train(
    model: nn.Module,
    dataloader: DataLoader,
    val_dataloader: DataLoader,
    n_epochs: int,
    optimizer: torch.optim.Optimizer,
    scheduler: torch.optim.lr_scheduler.CosineAnnealingLR,
    verbose: bool = False,
    contrastive_flow_weight: float = 0.05,
):
    def step(x_raw, t):
        x = to_latent(vae, x_raw)
        noise = torch.randn_like(x, device=DEVICE)
        t_expanded = t.unsqueeze(-1).unsqueeze(-1)
        z = t_expanded * x + (1 - t_expanded) * noise
        target_u = x - noise
        u = model(z, t.squeeze(-1))
        loss = F.mse_loss(u, target_u)
        if contrastive_flow_weight > 0.0:
            u_hat = torch.roll(u, shifts=1, dims=0)
            loss_contrastive = F.mse_loss(u, u_hat)
            loss = loss - contrastive_flow_weight * loss_contrastive
        return loss

    def get_fid(nfe: int = 25):
        real_images = next(iter(val_dataloader))[0]
        real_images = (real_images * 255).to(torch.uint8).to(DEVICE) # BS x 3 x 32 x 32
        z_s = sample_with_ode(model, n_samples=real_images.shape[0], n_steps=nfe)
        generated_images = from_latent(vae, z_s)
        generated_images = (generated_images * 255).to(torch.uint8).to(DEVICE) # BS x 3 x 32 x 32
        fid = FrechetInceptionDistance().to(DEVICE)
        fid.update(real_images, real=True)
        fid.update(generated_images, real=False)
        return fid.compute()

    def ckpt_callback(epoch: int, val_loss: float):
        ckpt_dir = "../ckpt"
        subdir = f"{ckpt_dir}/{model.__class__.__name__}"
        if not os.path.exists(subdir):
            os.makedirs(subdir)
        torch.save(model.state_dict(), f"{subdir}/model.pth")
        torch.save(optimizer.state_dict(), f"{subdir}/optimizer.pth")
        torch.save(scheduler.state_dict(), f"{subdir}/scheduler.pth")
        print(f"Saved checkpoint at epoch {epoch} with val loss {val_loss:.4f}")

    log_interval = 1
    model.train()
    best_val_loss = float("inf")
    loss_history = []
    val_loss_history = []
    fid_25_history = []
    fid_50_history = []
    for epoch in range(n_epochs):
        losses = []
        for x_raw,_, t in dataloader:
            loss = step(x_raw.to(DEVICE), t.to(DEVICE))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        scheduler.step()

        val_losses = []
        with torch.no_grad():
            for x_raw,_, t in val_dataloader:
                loss = step(x_raw.to(DEVICE), t.to(DEVICE))
                val_losses.append(loss.item())

        current_loss = float(np.mean(losses))
        current_val_loss = float(np.mean(val_losses))

        fid_25 = get_fid(nfe=25)
        fid_50 = get_fid(nfe=50)
        fid_25_history.append(fid_25)
        fid_50_history.append(fid_50)
        if (epoch % log_interval == 0 or epoch == n_epochs - 1) and verbose:
            print(f"Epoch {epoch}\t loss: {current_loss:.4f}\t val loss: {current_val_loss:.4f}\t FID_25: {fid_25:.4f}\t FID_50: {fid_50:.4f}")

        loss_history.append(current_loss)
        val_loss_history.append(current_val_loss)

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            ckpt_callback(epoch, current_val_loss)

    return loss_history, val_loss_history

In [ ]:
loss_history, val_loss_history = train(
    model=model,
    dataloader=train_dataloader,
    val_dataloader=test_dataloader,
    n_epochs=N_EPOCHS,
    optimizer=optimizer,
    scheduler=scheduler,
    verbose=True,
)
plt.plot(loss_history)
plt.plot(val_loss_history)
plt.show()

Epoch 0	 loss: 19.9613	 val loss: 16.8138	 FID_25: 275.2579	 FID_50: 282.5410
Saved checkpoint at epoch 0 with val loss 16.8138
Epoch 1	 loss: 14.9413	 val loss: 13.6745	 FID_25: 267.5249	 FID_50: 271.3111
Saved checkpoint at epoch 1 with val loss 13.6745
Epoch 2	 loss: 12.2915	 val loss: 11.4090	 FID_25: 266.2974	 FID_50: 264.9057
Saved checkpoint at epoch 2 with val loss 11.4090


In [ ]:
from torch.utils.data import TensorDataset


@torch.no_grad()
def generate_rectified_dataset(
    model_v0: nn.Module, original_dataset: torch.Tensor, n_steps=100, batch_size=512
):
    model_v0.eval()
    model_device = next(model_v0.parameters()).device
    t_span = torch.linspace(1.0, 0.0, n_steps).to(model_device)
    p1_data = original_dataset.to(model_device)

    p0_rectified_list = []
    for p1_batch in torch.split(p1_data, batch_size):
        p0_batch = odeint(
            model_v0.ode_forward,
            p1_batch,
            t_span,
            method="euler",
            atol=1e-5,
            rtol=1e-5,
        )[-1]
        p0_rectified_list.append(p0_batch)

    p0_rectified = torch.cat(p0_rectified_list, dim=0)
    return TensorDataset(p0_rectified.cpu(), original_dataset.cpu())


In [ ]:
N_EPOCHS_RF = 100
rf_model = Flow().to(DEVICE)
rectified_dataset = generate_rectified_dataset(model_v0=model.to("cpu"), original_dataset=data.X)
dataloader = DataLoader(rectified_dataset, batch_size=128, shuffle=True)
optimizer = AdamW(rf_model.parameters(), lr=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=N_EPOCHS_RF)

In [ ]:
loss_history = train(
    rf_model,
    dataloader,
    n_epochs=N_EPOCHS_RF,
    optimizer=optimizer,
    scheduler=scheduler,
    t_dist=time_distribution,
)

In [ ]:
general_plot(rf_model, title="Contrastive Flow", n_samples=1000, n_steps=20)

In [ ]:
general_plot(rf_model, title="Contrastive Flow", n_samples=1000, n_steps=20, animate=True)

In [ ]:
rf_model_2 = Flow().to(DEVICE)
rectified_dataset = generate_rectified_dataset(
    model_v0=rf_model, original_dataset=data.X
)
dataloader = DataLoader(rectified_dataset, batch_size=128, shuffle=True)
optimizer = AdamW(rf_model_2.parameters(), lr=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=N_EPOCHS_RF)

In [ ]:
loss_history = train(
    rf_model_2,
    dataloader,
    n_epochs=N_EPOCHS_RF,
    optimizer=optimizer,
    scheduler=scheduler,
    t_dist=time_distribution,
)

In [ ]:
general_plot(rf_model_2, title="Contrastive Flow (Reflow #2)", n_samples=1000, n_steps=5)

In [ ]:
general_plot(rf_model_2, title="Contrastive Flow (Reflow #2)", n_samples=1000, n_steps=5, animate=True)